In [1]:
import numpy as np
X = np.load('datasets/delivery_review.npy')
y = np.load('datasets/delivery_label.npy')

In [2]:
with open('datasets/chinese_stop_words.txt', encoding="utf-8") as text:
    stopwords = [line.strip() for line in  text]

In [3]:
import jieba
X_new = []
for i, review in enumerate(X):
    review = list(jieba.cut(review))
    result = []
    for word in review:
        if word not in stopwords:
            result.append(word)
    X_new.append(" ".join(result))
X_new = np.array(X_new)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BEIZHO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.883 seconds.
Prefix dict has been built succesfully.


In [4]:
X_new[1]

'分量 特色 下回 不吃'

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, shuffle=True)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

maxlen = max([len(sentence.split(" ")) for sentence in X_new])

words_size = 10000
tokenizer = Tokenizer(num_words=words_size)
tokenizer.fit_on_texts(X_new)

sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, 
                        maxlen=maxlen, 
                        padding='pre')

sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, 
                       maxlen=maxlen, 
                       padding='pre')

Using TensorFlow backend.


In [7]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Dense, Dropout
from keras.optimizers import Adam

embedding_dim = 32
model = Sequential()
model.add(Embedding(input_dim=words_size, 
                    input_length=maxlen,
                    output_dim=embedding_dim))

model.add(LSTM(units=256, recurrent_dropout=0.5))
model.add(Dropout(0.5))

model.add(Dense(units=1, 
                activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 135, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 616,193
Trainable params: 616,193
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,
          y_train,
          epochs=5,
          batch_size=32,
          validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Train on 5120 samples, validate on 1280 samples
Epoch 1/5
5120/5120 [==============================] - 67s 13ms/step - loss: 0.5526 - acc: 0.7287 - val_loss: 0.3853 - val_acc: 0.8281
Epoch 2/5
5120/5120 [==============================] - 62s 12ms/step - loss: 0.3390 - acc: 0.8557 - val_loss: 0.3585 - val_acc: 0.8539
Epoch 3/5
5120/5120 [==============================] - 62s 12ms/step - loss: 0.2902 - acc: 0.8873 - val_loss: 0.3899 - val_acc: 0.8375
Epoch 4/5
4672/5120 [==========================>...] - ETA: 4s - loss: 0.2079 - acc: 0.9232